In [1]:
import numpy as np
import pandas as pd
import psycopg2

import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.style
matplotlib.style.use('ggplot')
%matplotlib inline

In [2]:
# create a database connection
sqluser = 'mimic'
dbname = 'mimic'
schema_name = 'mimiciii'

cur = None

In [39]:
if cur: 
    cur.close()
    con.close()

con = psycopg2.connect(dbname = dbname, user = sqluser)
cur = con.cursor()
cur.execute('SET search_path to ' + schema_name)

In [42]:
query = \
"""
SELECT * FROM echo_features; 
"""
ef = pd.read_sql_query(query, con)
ef.shape

(25512, 26)

In [40]:
query = \
"""
SELECT * FROM echo_features_labs; 
"""
el = pd.read_sql_query(query, con)
el.head()

,row_id,labid,min_abs_dt
0,59653,50810,2 days 11:22:00
1,59653,50811,2 days 11:22:00
2,59653,50813,2 days 11:22:00
3,59653,50908,2 days 11:35:00
4,59653,50910,2 days 11:35:00


In [41]:
el['labid'].value_counts()

51301    25376
50912    25082
50820    21656
50813    19320
50862    18018
50910    17889
50911    16954
50811    14889
50810    14889
51003    13943
50920     7789
50908     6352
50963     3348
51288     2227
50889     1946
51002     1740
51300      296
Name: labid, dtype: int64

In [30]:
el.loc[el['row_id'] == 67394]

,row_id,lab_creatinine_charttime,lab_creatinine_dt,lab_creatinine_valuenum,lab_creatinine_valueuom,lab_creatinine_flag
3581,67394,2142-03-12 03:55:00,-1 days +12:16:00,1.5,mg/dL,abnormal
3582,67394,2142-03-13 03:23:00,11:44:00,1.1,mg/dL,None


In [34]:
el.loc[el['row_id'] == 64997]

,row_id,lab_creatinine_charttime,lab_creatinine_dt,lab_creatinine_valuenum,lab_creatinine_valueuom,lab_creatinine_flag
2464,64997,2127-12-05 17:07:00,05:06:00,8.6,mg/dL,abnormal
2465,64997,2127-12-05 06:55:00,-1 days +18:54:00,8.3,mg/dL,abnormal


In [38]:
el['row_id'].value_counts().value_counts()

1    25082
Name: row_id, dtype: int64

This is is really awkward. Basically, if there are multiple labs that meet the minimum dt, then this happens. One possibility is take the mean or something, but the flags may be conflicting. 

Number of unique hospital admission IDs with at least 1 lab: 

In [16]:
el[['label', 'hadm_id']].drop_duplicates().groupby('label')['hadm_id'].count()

label
Albumin                          1146
C-Reactive Protein                 81
CK-MB Index                       139
Creatine Kinase (CK)              759
Creatine Kinase, MB Isoenzyme     655
Creatinine                       1931
Estimated GFR (MDRD equation)     737
Hematocrit, Calculated           1580
Hemoglobin                       1580
Lactate                          1730
NTproBNP                           70
Sedimentation Rate                 79
Troponin I                         25
Troponin T                        544
WBC Count                          18
White Blood Cells                1930
pH                               1874
Name: hadm_id, dtype: int64

Look at the labs 
+ Hematocrit, Hemogloblin, WBC should be the same

Search lab terms in d_items: 

In [18]:
query = \
"""
SELECT * FROM d_items
WHERE label ~* '.*hematocrit.*'; 
"""
dil = pd.read_sql_query(query, con)
dil.head()

,row_id,itemid,label,abbreviation,dbsource,linksto,category,unitname,param_type,conceptid
0,732,813,Hematocrit,None,carevue,chartevents,Hematology,None,None,None
1,2878,3761,Hematocrit (35-51),None,carevue,chartevents,ABG'S,None,None,None
2,14694,227017,Hematocrit_ApacheIV,Hematocrit_ApacheIV,metavision,chartevents,Scores - APACHE IV (2),None,Numeric,None
3,12685,220545,Hematocrit (serum),Hematocrit (serum),metavision,chartevents,Labs,None,Numeric,None
4,14407,226540,Hematocrit (whole blood - calc),Hematocrit (whole blood - calc),metavision,chartevents,Labs,None,Numeric,None
